## Семинар 5

# Тема: Масштабирование (нормализация) данных. Применение конвейера.

Нормализация признаков - приведение признаков к одному масштабу. Нормализация признаков относится к методам предобработки данных. Её необходимо производить в связи с тем, что многие алгоритмы машинного обучения лучше работают, когда все признаки имеют одну шкалу измерения. Деревья и леса представляют собой два алгоритма из очень немногих, где не нужно масштабировать данные. Для таких алгоритмов, которые работают независимо от масштаба признаков, хуже от нормализации обычно не становится. Нормализация применяется только к числовым признакам. Существует целый ряд методов шкалирования. Рассмотрим некоторые из них.

## minmax нормализация

`minmax` нормализация приводит каждый признак к значению между 0 и 1 или от a до b.  
Признак `x` трансформируется по формуле: $${x_i}^{norm} := \frac{x_i - x_{min}}{x_{max} - x_{min}}$$

$${x_i}^{norm} := a+\frac{(x_i - x_{min})(b-a)}{x_{max} - x_{min}}$$

Класс `MinMaxScale`из библиотеки `sklearn` имеет метод `fit` для вычисления минимального ($x_{min}$) и максимального ($x_{max}$) значения признака, а метод `transform` преобразует признак по формуле. Можно выполнить обе этих операции сразу так `fit_transform`:

`from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))# по умолчанию диапазон от 0 до 1
scaler.fit_transform(X_train)
scaler.transform(X_test)`

## std нормализация (стандартная нормализация)

`std` нормализация приводит каждый признак к виду нормального распределения, когда он имеет среднее значение 0 и стандартное отклонение 1.  
Признак `x` трансформируется по формуле: $${x_i}^{st} := \frac{x_i - x_{mean}}{x_{std}}$$

Класс `StandardScaler`из библиотеки `sklearn` имеет метод `fit` для вычисления среднего значения ($x_{mean}$) признака и его стандартного отклонения ($x_{std}$), а метод `transform` преобразует признак по формуле. Можно выполнить обе этих операции сразу так `fit_transform`:

`from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit_transform(X_train)
scaler.transform(X_test)`

Нормализация min-max предпочтительна, когда данные не соответствуют гауссовскому (нормальному) распределению. Нормализация min-max предпочтительна для алгоритмов, которые не следуют какому-либо распределению. Обратите внимание, что на min-max нормализацию влияют выбросы.

С другой стороны, стандартная нормализация может быть полезна в тех случаях, когда данные подчиняются распределению Гаусса. Однако это не обязательно. Кроме того, в отличие от min-max нормализации, стандартная нормализация не имеет ограничивающего диапазона. Это означает, что даже если в данных есть выбросы, стандартизация на них не повлияет.

### Применение конвейера

Класс Pipeline позволяет писать более лаконичный код и уменьшает вероятность ошибок, которые могут быть допущены при построении цепочек операций без использования класса Pipeline (например, мы можем забыть применить все преобразования к тестовому набору или можем применить их в неправильном порядке).

Класс make_pipeline - конвейер c автоматическим присваиванием имени каждому этапу.

Импортируем необходимые библиотеки:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing, load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline

### Задание: 
Загрузите данные о заболевании диабетом. Масштабируйте их при помощи min-max нормализации. Обучите модель линейной регрессии на этих данных. Выведите коэффициенты гиперплоскости и метрики качества. Проделайте то же самое, используя конвейер.

Загружаем данные:

In [2]:
diabetes = load_diabetes()
X=housing.data
y=housing.target
df = pd.DataFrame(X, columns = diabetes.feature_names)
df['y'] = y
df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,y
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


Разделяем на обучающую и тестовую выборки:

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

Масштабируем обучающие и тестовые данные:

In [4]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Обучаем модель на масштабированных данных:

In [5]:
model_linreg = LinearRegression()
model_linreg.fit(X_train_scaled, y_train)

LinearRegression()

Выводим коэффициенты:

In [6]:
model_linreg.coef_

array([  -9.42905024,  -19.89039971,  154.77853961,   74.03941014,
       -147.22209812,   82.18680495,   -2.51359979,   35.56367877,
        182.61282974,    7.75023385])

In [7]:
model_linreg.intercept_

8.133812537354288

Делаем предсказание на тестовых данных:

In [8]:
y_test_pred = model_linreg.predict(X_test_scaled)

Вычислим коэффициент детерминации:

In [9]:
r2 = metrics.r2_score(y_test,y_test_pred)
r2

0.35940880381777107

In [10]:
RMSE = metrics.root_mean_squared_error(y_test,y_test_pred)
RMSE

56.39290423586857

### Испортируем класс Pipeline.

Задаем названия и создаем объекты используемых классов.

In [11]:
pipe = Pipeline(steps = [('scaler', MinMaxScaler()),
                         ('lr', LinearRegression())])

Шаги конвейера:

In [12]:
pipe.steps

[('scaler', MinMaxScaler()), ('lr', LinearRegression())]

Обучаем модель:

In [13]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', MinMaxScaler()), ('lr', LinearRegression())])

Выводим коэффициенты:

In [14]:
pipe.named_steps['lr'].intercept_

8.133812537354288

In [15]:
pipe.named_steps['lr'].coef_

array([  -9.42905024,  -19.89039971,  154.77853961,   74.03941014,
       -147.22209812,   82.18680495,   -2.51359979,   35.56367877,
        182.61282974,    7.75023385])

Сделаем предсказание на тестовых данных. Вычислим метрики:

In [16]:
y_test_pred=pipe.named_steps['lr'].predict(pipe.named_steps['scaler'].transform(X_test))

In [17]:
r2 = metrics.r2_score(y_test,y_test_pred)
r2

0.35940880381777107

In [18]:
RMSE = metrics.root_mean_squared_error(y_test,y_test_pred)
RMSE

56.39290423586857

### 1. Загрузите встроенные данные о показателе медианной стоимости дома в округах Калифорнии. Выведите описание датасета. Обозначьте за _X_ данные, а за _y_ целевую переменную. Выведите их размеры.

In [3]:
houses = fetch_california_housing()

In [8]:
print(houses.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

In [5]:
X = houses.data
X

array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]])

In [6]:
y = houses.target

In [9]:
X.shape

(20640, 8)

In [10]:
y.shape

(20640,)

### 2. Разделите данные на обучающую и тестовую части. Выведите их размеры. 

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [15]:
X_train.shape

(15480, 8)

In [16]:
X_test.shape

(5160, 8)

### 3. Постройте модель линейной регрессии для предсказания медианной стоимости дома в округах Калифорнии. Выведите, полученные коэффициенты  гиперплоскости. Сделайте предсказание на тестовых данных. Вычислите метрики: коэффициент детерминации и ошибка RMSE. Сделайте вывод о качестве работы модели линейной регрессии для этих данных. 

In [17]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

### 4. Напишите своими руками функцию MinMaxScaler_fit, которая принимает матрицу признаков, а возвращает массив минимальных значений по каждому признаку и массив максимальных значений по каждому признаку. Примените её к обучающим данным.

### 5. Напишите своими руками функцию _MinMaxScaler_transform_, которая принимает матрицу признаков, а также numpy-массивы минимальных и максимальных значений и возвращает масштабированные признаки. Масштабируйте обучающие и тестовые данные при помощи этой функции. 
Указание: ${x_i}^{norm} := \frac{x_i - x_{min}}{x_{max} - x_{min}}$. Примените эту функцию к обучающим и тестовым данным, передав в неё массивы минимальных и максимальных значений обучающих данных.

### 6. Убедитесь, что у масштабированных обучающих данных минимальное значение стало равно 0, а максимальное 1. А у масштабированных тестовых данных минимальное значение стало близко к 0, а максимальное близко к 1.

### 7. Осуществите масштабирование признаков методом min-max при помощи библиотечной функции.
Указание: Создайте объект класса [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html). Примените к нему метод _fit_, передав в него обучающие данные, он рассчитает минимальное и максимальное значение каждого из столбцов обучающих данных. Их можно посмотреть через соответствующие атрибуты: _data_min__ и _data_max__.  Далее при помощи метода _transform_ преобразуйте обучающие и тестовые данные. Убедитесь, что они будут теми же, что и при шкалировании вручную. Для обучающих данных можно сразу применять метод _fit_transform_.

### 8. Изобразите гистограммы признаков до и после min-max масштабирования.

### 9. Обучите модель линейной регрессии для предсказания медианной стоимости дома в округах Калифорнии на масштабированных данных методом min-max. Выведите полученные коэффициенты  гиперплоскости. Напечатайте рядом название признака и соответствующий ему вес. Сделайте вывод о значимости признаков. 

### 10. Сделайте предсказание на тестовых данных. Вычислите метрики: коэффициент детерминации и ошибка RMSE. Убедитесь, что значения метрик практически те же, что и при обучении модели на не масштабированных данных.

### 11. Напишите своими руками функцию _StandardScaler_fit_, которая принимает матрицу признаков, а возвращает массив средних значений по каждому признаку и массив стандартных отклонений по каждому признаку. Примените её к обучающим данным.

### 12. Напишите своими руками функцию _StandardScaler_transform_, которая принимает матрицу признаков, а также numpy-массивы средних значений и стандартных отклонений и возвращает стандартизированные признаки. Стандартизируйте обучающие и тестовые данные. 
Указание: ${x_i}^{norm} := \frac{x_i - x_{mean}}{x_{std}}$. Примените эту функцию к обучающим и тестовым данным, передав в неё массивы средних значений и стандартных отклонений обучающих данных.

### 13. Убедитесь, что у стандартизированных обучающих данных среднее значение стало равно 0, а стандартное отклонение равно 1. А у масштабированных тестовых данных среднее значение стало близко к 0, а стандартное отклонение близко к 1.

### 14. Осуществите стандартизацию признаков при помощи библиотечных функций.
Указание: Создайте объект класса [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html). Примените к нему метод _fit_, передав в него обучающие данные, он рассчитает среднее арифметическое и среднее квадратическое отклонение каждого из столбцов обучающих данных. Их можно посмотреть через соответствующие атрибуты: _mean__ и _scale__.  Далее при помощи метода _transform_ преобразуйте обучающие и тестовые данные. Убедитесь, что они будут теми же, что и при стандартизации вручную. Для обучающих данных можно сразу применять _fit_transform_.

### 15. Изобразите гистограммы признаков до и после стандартизации.

### 16. Обучите модель линейной регрессии для предсказания медианной стоимости дома в округах Калифорнии на стандартизированных данных. Выведите полученные коэффициенты  гиперплоскости. Напечатайте рядом название признака и соответствующий ему вес. Сделайте вывод о значимости признаков. 

### 17. Обучите модель линейной регрессии на стандартизированных данных с применением конвейера, используя класс Pipeline. Выведите полученные коэффициенты гиперплоскости. 

### 18. Сделайте предсказание на тестовых данных. Вычислите метрики: коэффициент детерминации и ошибка RMSE. Убедитесь, что они получились те же, что и без применения конвейера.

### 19. Обучите модель линейной регрессии на стандартизированных данных с применением конвейера, используя класс make_pipeline. Выведите полученные коэффициенты гиперплоскости. Сделайте предсказание на тестовых данных и вычислите метрики.